# Gas Station_주유소

- 주유소 : `Resource`
- 공유 연료 탱크 : `Container`
- 다른 프로세스 종료될 때까지 기다리는 시스템

`주유소`와 주유를 위해 주유소에 도착하는 `자동차`로 구성되어 있는 시뮬레이션 환경을 생성합니다.

주유소에는 제한된 수의 `연료 펌프`와 연료 펌프 간에 공유되는 `연료 탱크`가 있습니다. 따라서 주유소는 `Resource`로 모델링됩니다. 공유 연료 탱크는 `Container`로 모델링됩니다.

주유소에 도착하는 차량들은 먼저 주유소에 연료 펌프를 요청합니다. 일단 연료 펌프를 얻으면, 그들은 연료 펌프에서 원하는 양의 연료를 얻으려고 합니다. 차량은 주유가 끝나면 주유소를 떠난다.

주유소 연료 레벨은 주유소 제어에 의해 정기적으로 모니터링됩니다. 수위가 일정 임계치 이하로 떨어지면 탱크트럭을 불러 주유소 자체에 주유를 하게 됩니다.

In [1]:
import itertools
import random
import simpy

RANDOM_SEED = 42
GAS_STATION_SIZE = 200     # 주유소 리터수
THRESHOLD = 10             # 탱크트럭을 불러 주유소에 주유하는 임계치
FUEL_TANK_SIZE = 50        # 차량의 연료 탱크 리터수
FUEL_TANK_LEVEL = [5, 25]  # 차량의 현재 연료 탱크 존재하는 리터수. [최대/최소] 사이 존재
REFUELING_SPEED = 2        # (리터/초)단위의 주유 속도
TANK_TRUCK_TIME = 300      # 탱크 트럭이 도착하는 시간(초)
T_INTER = [30, 300]        # 차량이 생성되는 시간 [최소, 최대] 시간(초) 사이로 생성됨
SIM_TIME = 1000            # 시뮬레이션 전체 시간(초)

In [7]:
def car(name, env, gas_station, fuel_pump):
    """주유하기 위해 주유소에 도착하는 차량 함수."""
    fuel_tank_level = random.randint(*FUEL_TANK_LEVEL) # 
    print('(%.1f초) %s 이 주유소 도착' % (env.now, name))
    with gas_station.request() as req:
        start = env.now
        yield req # 가스 펌프 중 하나 요청
        # 차량의 연료 탱크 크기에서 현재 차량의 연료 레벨의 차만큼 필요함
        liters_required = FUEL_TANK_SIZE - fuel_tank_level
        yield fuel_pump.get(liters_required)
        # 필요한 양만큼 주유하는 동안 걸리는 시간
        yield env.timeout(liters_required / REFUELING_SPEED)
        print('(%.1f초) %s 이 %.1f 초만에 주유를 마쳤습니다.' % (env.now ,name, env.now - start))

In [8]:
def gas_station_control(env, fuel_pump):
    """주기적으로 주유소 자체의 연료 탱크를 확인한 후 임계치 이하로 떨어지면 탱크 트럭 호출"""
    while True:
        if fuel_pump.level / fuel_pump.capacity * 100 < THRESHOLD:
            # 탱크 트럭을 불러야 합니다.
            print('(%d초) 탱크 트럭 호출이 필요합니다.' % env.now)
            # 탱크 트럭이 도착하여 주유소에 연료를 채우는 동안 기다려야 합니다.
            yield env.process(tank_truck(env, fuel_pump))

        yield env.timeout(10)  # 10초마다 확인


In [9]:
def tank_truck(env, fuel_pump):
    """탱크 트럭이 도착하여 주유소에 연료 탱크에 급유"""
    yield env.timeout(TANK_TRUCK_TIME)
    print('(%d초) 탱크 트럭 도착' % env.now)
    ammount = fuel_pump.capacity - fuel_pump.level
    print('탱크 트럭이 주유소에 %.1f 리터만큼 급유.' % ammount)
    yield fuel_pump.put(ammount)

In [10]:
def car_generator(env, gas_station, fuel_pump):
    """주유소에 도착하는 차량 랜덤 생성"""
    for i in itertools.count():
        yield env.timeout(random.randint(*T_INTER))
        env.process(car('차량 %d' % i, env, gas_station, fuel_pump))

In [11]:
# Setup and start the simulation
print('########### 주유소 시뮬레이션 시작! ##############')
random.seed(RANDOM_SEED)

# Create environment and start processes
env = simpy.Environment()
gas_station = simpy.Resource(env, 2)
fuel_pump = simpy.Container(env, GAS_STATION_SIZE, init=GAS_STATION_SIZE)
env.process(gas_station_control(env, fuel_pump))
env.process(car_generator(env, gas_station, fuel_pump))

# Execute!
env.run(until=SIM_TIME)

########### 주유소 시뮬레이션 시작! ##############
(87.0초) 차량 0 이 주유소 도착
(105.5초) 차량 0 이 18.5 초만에 주유를 마쳤습니다.
(129.0초) 차량 1 이 주유소 도착
(148.0초) 차량 1 이 19.0 초만에 주유를 마쳤습니다.
(284.0초) 차량 2 이 주유소 도착
(305.0초) 차량 2 이 21.0 초만에 주유를 마쳤습니다.
(385.0초) 차량 3 이 주유소 도착
(398.5초) 차량 3 이 13.5 초만에 주유를 마쳤습니다.
(459.0초) 차량 4 이 주유소 도착
(460초) 탱크 트럭 호출이 필요합니다.
(481.0초) 차량 4 이 22.0 초만에 주유를 마쳤습니다.
(705.0초) 차량 5 이 주유소 도착
(750.0초) 차량 6 이 주유소 도착
(760초) 탱크 트럭 도착
탱크 트럭이 주유소에 188.0 리터만큼 급유.
(779.0초) 차량 6 이 29.0 초만에 주유를 마쳤습니다.
(781.5초) 차량 5 이 76.5 초만에 주유를 마쳤습니다.
(891.0초) 차량 7 이 주유소 도착
(904.0초) 차량 7 이 13.0 초만에 주유를 마쳤습니다.
